# Evaluation Python avancé

## Partie 1 : Algorithme de Needleman-Wunsch

In [1]:
from colorama import Fore, Style

def red_text(text):
    return f"{Fore.RED}{text}{Style.RESET_ALL}" # permet d'afficher du texte en rouge

### Classe Ruler

In [2]:
class Ruler :
    """classe d'objets constitués de 2 chaînes de caractères pouvant être comparées entre elles"""
    def __init__(self, chaine_1, chaine_2) :
        self.chaine_1 = chaine_1
        self.chaine_2 = chaine_2
        self.matrice = [[]]
        self.alignement_1 = ''
        self.alignement_2 = ''
    
    def compute(self) :
        """cette méthode  permet de déterminer les alignements obtenus à partir des chaînes de caractères du Ruler qui minimisent la distance"""
        chaine_1 = self.chaine_1
        chaine_2 = self.chaine_2
        a = len(chaine_1)
        b = len(chaine_2)
        s = [[1 for j in range(b)] for i in range(a)] # s est la matrice des coûts : une insertion, substitution ou suppression a un coût de 1
        for i in range(a) :
            for j in range(b) :
                if chaine_1[i] == chaine_2[j] :
                    s[i][j] = 0 #si deux lettres sont identiques : coût de 0
        F = [[0 for j in range(b+1)] for i in range(a+1)] #la matrice F est la matrice des scores : c'est elle qui va permettre de déterminer l'alignement de distance minimale 
        for i in range(a+1) :
            F[i][0] = i
        for j in range(b+1) :
            F[0][j] = j
        for i in range(1, a+1) :
            for j in range(1, b+1) :
                F[i][j] = min(F[i-1][j-1] + s[i-1][j-1], F[i][j-1] + 1, F[i-1][j] + 1) #le premier terme est le score d'une substitution, le deuxième celui d'une insertion, le dernier celui d'une suppression
        alignement_1 = ''
        alignement_2 = ''
        i = a
        j = b
        while i > 0 and j > 0 : # on "remonte" la matrice F pour déterminer l'alignement de distance minimale
            score = F[i][j]
            score_up = F[i-1][j]
            score_left = F[i][j-1]
            score_diag = F[i-1][j-1]
            if score == score_up + 1 : #suppression de la lettre chaine_1[i-1] dans chaine_2
                alignement_1 = chaine_1[i-1] + alignement_1
                alignement_2 = '-' + alignement_2
                i = i-1
            elif score == score_left + 1 : #insertion dans chaine_2 d'une nouvelle lettre
                alignement_1 = '-' + alignement_1
                alignement_2 = chaine_2[j-1] + alignement_2
                j = j-1
            else : # substitution
                alignement_1 = chaine_1[i-1] + alignement_1
                alignement_2 = chaine_2[j-1] + alignement_2
                j = j-1
                i = i-1
        while i >= 1 : #si on a finit de parcourir la chaîne 2 mais qu'il rest des lettres de la chaîne 1
            alignement_1 = chaine_1[i-1] + alignement_1
            alignement_2 = '-' + alignement_2
            i = i-1
        while j >= 1 : # c'est l'inverse : si la chaîne 1 est plus petite que la chaîne 2
            alignement_1 = '-' + alignement_1
            alignement_2 = chaine_2[j-1] + alignement_2
            j = j-1
        self.matrice = F
        self.alignement_1 = alignement_1
        self.alignement_2 = alignement_2
    
    def distance(self) :
        """la distance est directement donnée par le terme en bas à droite de la matrice : on somme successivement les coûts minimaux"""
        return self.matrice[len(self.chaine_1)][len(self.chaine_2)]
    
    def report(self) :
        """cette fonction permet d'afficher les deux alignements obtenus, en metant en rouge les différences, de façon à faciliter leur comparaison"""
        alignement_1 = self.alignement_1
        alignement_2 = self.alignement_2
        a1 = ''
        a2 = ''
        for i in range(len(alignement_1)) :
            a = alignement_1[i]
            b = alignement_2[i]
            if a != b :
                a1 = f"{a1}{red_text(a)}"
                a2 = f"{a2}{red_text(b)}"
            else :
                a1 = f"{a1}{a}"
                a2 = f"{a2}{b}"
        print (a1, a2)

#### Test de la classe

In [22]:
c1 = "abcd"
c2 ='acd'
c3 ='aabcd'   
ruler = Ruler(c2, c3)
ruler.compute()

In [23]:
print(f"distance = {ruler.distance()}")

distance = 2


In [24]:
ruler.report()

a--cd aabcd


### Module Bundle

In [7]:
fichier = open(r'C:\Users\Jeanne\Documents\DATASET.txt', 'r')
contenu = fichier.read()

In [9]:
L = []
with open(r'C:\Users\Jeanne\Documents\DATASET.txt', 'r') as fichier :
    for line in fichier :
        L.append(line.strip())
    print(L)
n = len(L)
for k in range(0, n - 1, 2) :
    ruler = Ruler(L[k], L[k + 1])
    ruler.compute()
    print(f"=== exemple #{k//2 + 1} - distance = {ruler.distance()}")
    ruler.report()

['abcdefg', 'qbcdefh', 'ACGT', 'ACCT', 'ade']
=== exemple #1 - distance = 2
abcdefg qbcdefh
=== exemple #2 - distance = 1
ACGT ACCT


In [10]:
from argparse import ArgumentParser
#from needlemann_wunsch import Ruler
parser = ArgumentParser()
parser.add_argument("fichier", help = "le nom du fichier")
args = parser.parse_args()

with open(args.fichier, 'r') as f :
    L = []
    for line in f :
        L.append(line.strip())
    n = len(L)
for k in range(0, n - 1, 2) :
    ruler = Ruler(L[k], L[k + 1])
    ruler.compute()
    print(f"=== exemple #{k//2 + 1} - distance = {ruler.distance()}")
    ruler.report()

usage: ipykernel_launcher.py [-h] fichier
ipykernel_launcher.py: error: unrecognized arguments: -f


SystemExit: 2

c:\users\jeanne\miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## Partie 2 : Codage de Huffman

In [14]:
class TreeBuilder :
    def __init__(self, text) :
        self.text = text
    
    def tree(self) :
        text = self.text
        occurences = {a : text.count(a) for a in text} #dictionnaire ou liste
        lettres = sorted(occurences.items(), key = lambda t : t[1])
        lettres = [x for (x,y) in lettres] # liste des lettres triée par ordre croissant d'occurrences
        m = occurences[lettres[0]] # arbre : [[gauche : 0],[droite : 1 ],'abc', 26] par ex
        ligne_sup = []
        while m < len(text) :
            ligne_inf = ligne_sup[:]
            ligne_sup=[]
            if lettres != [] :
                a = lettres.pop(0)
                x = occurences[a]
                while x <= m and lettres != [] :
                    ligne_inf.append([[],[], a, x])
                    a = lettres.pop(0)
                    x = occurences[a]
                if len(ligne_inf)%2 == 1 :
                    ligne_inf.append([[], [], a, x])
                else :
                    lettres = [a] + lettres   # à vérifier
            long = len(ligne_inf)
            if long > 1 :
                for i in range(0, long - 1, 2) :
                    gauche = ligne_inf[i]
                    droite = ligne_inf[i+1]
                    ligne_sup.append([gauche, droite, gauche[2] + droite[2], gauche[3] + droite[3]])
                if long%2 == 1 :
                    ligne_sup.append(ligne_inf[-1])
            m = max([l[3] for l in ligne_sup])
        return ligne_sup[0]

In [15]:
class Codec :
    def __init__(self, tree) :
        self.tree = tree
    
    def encode(self, text) :
        res = ''
        tree = self.tree
        for a in text :
            if a not in tree[2] :
                return 'incodable'
            else :
                L = tree[:]
                while len(L[2]) > 1 :
                    gauche, droite = L[0], L[1]
                    if a in gauche[2] :
                        res = res + '0'
                        L = L[0]
                    else :
                        res = res + '1'
                        L = L[1]
        return res
    
    def decode(self, code) :
        res = ''
        tree = self.tree
        k = 0
        L = tree[:]
        while k < len(code) :
            i = eval(code[k])
            L = L[i]
            if len(L[2]) == 1 :
                res += (L[2])
                L = tree[:]
            k += 1
        return res

In [18]:
text = 'on vérifie que cet algorithme marche'
arbre = TreeBuilder(text)
binary_tree = arbre.tree()
print(f"binary_tree = {binary_tree}")

binary_tree = [[[[[[[[], [], 'n', 1], [[], [], 'v', 1], 'nv', 2], [[[], [], 'é', 1], [[], [], 'f', 1], 'éf', 2], 'nvéf', 4], [[[[], [], 'q', 1], [[], [], 'u', 1], 'qu', 2], [[[], [], 'l', 1], [[], [], 'g', 1], 'lg', 2], 'qulg', 4], 'nvéfqulg', 8], [[[[], [], 'o', 2], [[], [], 'c', 2], 'oc', 4], [[[], [], 't', 2], [[], [], 'a', 2], 'ta', 4], 'octa', 8], 'nvéfqulgocta', 16], [[[[[], [], 'h', 2], [[], [], 'm', 2], 'hm', 4], [[], [], 'r', 3], 'hmr', 7], [[[], [], 'i', 3], [[], [], ' ', 5], 'i ', 8], 'hmri ', 15], 'nvéfqulgoctahmri ', 31], [[], [], 'e', 5], 'nvéfqulgoctahmri e', 36]


In [20]:
codec = Codec(binary_tree)
encoded = codec.encode(text)
decoded = codec.decode(encoded)
print (f"texte codé = {encoded}")
print(f"texte d'origine = {text}")
print(f"texte décodé = {decoded}")

texte codé = 0010000000001110000010000100101011000001101101011100010000010110111001011001100111001110001100001110010001010110001100100001001101110100100111010100101010001
texte d'origine = on vérifie que cet algorithme marche
texte décodé = on vérifie que cet algorithme marche


In [21]:
text == decoded

True